In [1]:
from cloudvolume import CloudVolume
from skimage.transform import downscale_local_mean
import napari
from skimage import io
import random
import h5py
from skimage import measure
from brainlit.preprocessing import removeSmallCCs
import numpy as np 
%gui qt5

/Users/thomasathey/Documents/mimlab/mouselight/env/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


# Download benchmark data

In [2]:

dir = "precomputed://https://dlab-colm.neurodata.io/2021_04_08/gad2cre_tph2flp_con_fon_8291/561"
mip = 0

vol_bg = CloudVolume(dir, parallel=1, mip=mip, fill_missing=True)
shape = vol_bg.shape
print(shape)

dir = "precomputed://https://dlab-colm.neurodata.io/2021_04_08/gad2cre_tph2flp_con_fon_8291/642"
mip = 0

vol_fg = CloudVolume(dir, parallel=1, mip=mip, fill_missing=True)
shape = vol_fg.shape
print(shape)

[ 7403 10240  4800     1]
[ 7403 10240  4800     1]


In [ ]:
centers_train = [[2477, 3638, 2409], [3605, 2873, 2405], [4939, 5186, 2398], [4538, 5148, 2398], [4618, 3225, 2388], [3223, 5206, 3550], [1953, 2102, 3577], [2395, 5004, 3584], [941, 3711, 708], [2030, 2164, 701], [3283, 3406, 1255], [1531, 2220, 1242], [2569, 6420, 2924], [2282, 8206, 2924], [4424, 5689, 2896], [3269, 3987, 2896], [2817, 6831, 4565], [3308, 3276, 4124], [4560, 6354, 4133], [4293, 2411, 2297]]
centers_val = [[5701, 3357, 4137], [3610, 2346, 4137], [2564, 4086, 2829], [1282, 2182, 2829], [3960, 2836, 1546], [2347, 4866, 1545], [1680, 4284, 806], [3223, 3294, 2514], [3564, 3847, 2516], [2620, 7237, 4610]]

In [7]:
for i, center in enumerate(centers_val):
    image_fg = vol_fg[center[0]-49:center[0]+50,center[1]-49:center[1]+50, center[2]-49:center[2]+50]
    image_fg = image_fg[:,:,:,0]

    image_bg = vol_bg[center[0]-49:center[0]+50,center[1]-49:center[1]+50, center[2]-49:center[2]+50]
    image_bg = image_bg[:,:,:,0]
    image_2channel = np.stack([image_bg, image_fg], axis=0)
    
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/benchmark_formal/val_" + str(i) + ".h5"
    with h5py.File(fname, "w") as f:
        dset = f.create_dataset("image_2channel", data=image_2channel)
    

Downloading: 390it [00:06, 64.52it/s] 
Downloading: 386it [00:06, 64.16it/s] 
Downloading: 784it [00:11, 67.37it/s] 
Downloading: 790it [00:13, 59.30it/s] 
Downloading: 790it [00:11, 68.40it/s] 
Downloading: 782it [00:11, 68.58it/s] 
Downloading: 788it [00:13, 57.69it/s] 
Downloading: 786it [00:11, 70.04it/s] 
Downloading: 786it [00:11, 69.29it/s] 
Downloading: 790it [00:11, 67.85it/s] 
Downloading: 780it [00:11, 68.16it/s] 
Downloading: 786it [00:11, 67.59it/s] 
Downloading: 386it [00:05, 66.10it/s] 
Downloading: 392it [00:07, 55.09it/s]
Downloading: 788it [00:11, 68.43it/s] 
Downloading: 790it [00:11, 68.40it/s] 
Downloading: 790it [00:11, 68.63it/s] 
Downloading: 790it [00:12, 65.80it/s] 
Downloading: 196it [00:03, 62.06it/s] 
Downloading: 186it [00:03, 60.61it/s]


## View data

In [51]:
center = centers_train[0]

image_fg = vol_fg[center[0]-49:center[0]+50,center[1]-49:center[1]+50, center[2]-49:center[2]+50]
image_fg = image_fg[:,:,:,0]

image_bg = vol_bg[center[0]-49:center[0]+50,center[1]-49:center[1]+50, center[2]-49:center[2]+50]
image_bg = image_bg[:,:,:,0]

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(image_fg)
viewer.add_image(image_bg)

Downloading: 392it [00:05, 74.46it/s] 
Downloading: 392it [00:04, 78.90it/s] 


<Image layer 'image_bg' at 0x1657a5d90>

## Check training results

In [65]:
for i in range(20):
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/benchmark_formal/train_" + str(i) + ".h5"
    f = h5py.File(fname, 'r')
    im = f.get('image_2channel')
    im_bg = im[0,:,:,:]
    im_fg = im[1,:,:,:]

    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/benchmark_formal/train_" + str(i) + "-image_2channel_Labels.h5"
    f = h5py.File(fname, 'r')
    gt = f.get('exported_data')
    gt = gt[0,:,:,:]
    pos_labels = gt == 2

    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/benchmark_formal/train_" + str(i) + "-image_2channel_Probabilities.h5"
    f = h5py.File(fname, 'r')
    seg = f.get('exported_data')
    seg = seg[1,:,:,:]
    mask = seg > 0.5
    
    true_pos = np.sum(np.logical_and(mask, pos_labels))
    precision = true_pos/np.sum(mask)
    recall = true_pos/np.sum(pos_labels)
    print(f"Example {i}: precision: {precision}, recall: {recall}")

    '''
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(im_bg)
    viewer.add_image(im_fg)
    viewer.add_labels(gt)
    viewer.add_labels(mask)
    '''

Example 0: precision: 0.005227442449430176, recall: 1.0
Example 1: precision: 0.0, recall: nan
Example 2: precision: 0.0, recall: nan
Example 3: precision: 0.0, recall: nan
Example 4: precision: nan, recall: nan
Example 5: precision: 0.0, recall: nan
Example 6: precision: 0.0034653465346534654, recall: 1.0
Example 7: precision: 0.00260778859527121, recall: 1.0
Example 8: precision: nan, recall: nan
Example 9: precision: nan, recall: nan
Example 10: precision: 0.0005051940260806416, recall: 1.0
Example 11: precision: nan, recall: nan
Example 12: precision: 0.0107095046854083, recall: 1.0
Example 13: precision: nan, recall: nan
Example 14: precision: 0.0, recall: nan
Example 15: precision: 0.0, recall: nan
Example 16: precision: nan, recall: nan
Example 17: precision: nan, recall: nan
Example 18: precision: nan, recall: nan
Example 19: precision: 0.029491525423728814, recall: 1.0


# Old

# Evaluate results


### Read ilastik

In [8]:
base_path = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/benchmark/"

part = "4"

im_path  = base_path + part + ".tif"
image = io.imread(im_path)

pred_path = base_path + part + "_Probabilities_3d_2channel.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
print(pred.shape)
pred = pred[1,:,:,:]
mask = pred > 0.5
mask = removeSmallCCs(mask, 100)
print(f"sum: {np.sum(mask)}")

(2, 200, 200, 200)
sum: 744


### Read swcs

In [3]:
from brainlit.utils.Neuron_trace import NeuronTrace
from pathlib import Path
from scipy.ndimage import distance_transform_edt

In [4]:
swc_path = Path("/Users/thomasathey/Documents/mimlab/mouselight/ailey/" + part + "_traces/")

swc_files = list(swc_path.glob("**/*.swc"))

paths_total = []
for swc_num, swc in enumerate(swc_files):

    swc_trace = NeuronTrace(path=str(swc))
    paths = swc_trace.get_paths()
    offset_diff, _, _, _ = swc_trace.get_df_arguments()

    for path_num, p in enumerate(paths):
        paths_total.append(p)

In [5]:
swc_mask = 0*mask
for path in paths_total:
    path = path.astype(int)
    swc_mask[path[:,0], path[:,1], path[:,2]] = 1
    
edt = distance_transform_edt(swc_mask==0)
swc_mask[edt < 2] = 1

In [10]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(np.swapaxes(image,0,2))
viewer.add_labels(np.swapaxes(mask,0,2))
#viewer.add_labels(swc_mask)
#viewer.add_shapes(data=paths_total, shape_type='path', edge_width=1.0, edge_color='blue', opacity=0.8)

<Labels layer 'Labels' at 0x1641747c0>

In [ ]:
tpr = np.sum(np.logical_and(swc_mask, np.swapaxes(mask,0,2)))/np.sum(swc_mask)
fpr = np.sum(np.logical_and(swc_mask==0, np.swapaxes(mask==1,0,2)))/np.sum(swc_mask==0)
print(f"TPR: {tpr}, FPR: {fpr}")

# Check blocks
## Download data

In [18]:
block_num = 500
with open("/Users/thomasathey/Documents/mimlab/mouselight/ailey/wholebrain_results/coords_1_unique.txt") as fp:
    counter = 0
    for line in fp:
        if counter == block_num:
            words = line.split()
            corner = [int(word) for word in words]
        counter += 1

dims = [200,200,600]
image_fg = vol_fg[corner[0]:corner[0]+dims[0], corner[1]:corner[1]+dims[1], corner[2]:corner[2]+dims[2]]
image_bg = vol_bg[corner[0]:corner[0]+dims[0], corner[1]:corner[1]+dims[1], corner[2]:corner[2]+dims[2]]


Downloading: 4796it [01:07, 71.27it/s]  
Downloading: 4796it [01:05, 73.19it/s]  


In [19]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(image_fg[:,:,:,0])
viewer.add_image(image_bg[:,:,:,0])
napari.run()

In [28]:
from random import sample
import numpy as np 
from tqdm import tqdm 

s = sample(list(np.arange(0, 659)), 10)
dims = [200,200,600]

for block_num in s:
    with open("/Users/thomasathey/Documents/mimlab/mouselight/ailey/wholebrain_results/coords_1_unique.txt") as fp:
        counter = 0
        for line in fp:
            if counter == block_num:
                words = line.split()
                corner = [int(word) for word in words]
            counter += 1
    upper_bd = [np.amin([corner[i]+dims[i], vol_fg.shape[i]]) for i in range(3)]
    image_fg = vol_fg[corner[0]:upper_bd[0], corner[1]:upper_bd[1], corner[2]:upper_bd[2]]
    image_bg = vol_bg[corner[0]:corner[0]+dims[0], corner[1]:corner[1]+dims[1], corner[2]:corner[2]+dims[2]]
    image_2channel = np.stack([image_bg, image_fg], axis=0)
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/wholebrain_results/images/on_" + str(block_num) + ".h5"
    with h5py.File(fname, "w") as f:
        dset = f.create_dataset("image_2channel", data=image_2channel)


Downloading: 7194it [01:45, 67.98it/s]  
Downloading: 7198it [01:39, 72.00it/s]  
Downloading: 7198it [01:40, 71.44it/s]  
Downloading: 7192it [01:33, 76.79it/s]  
Downloading: 7192it [01:39, 72.04it/s]  
Downloading: 7198it [01:42, 70.31it/s]  
Downloading: 4798it [00:55, 87.11it/s]  
Downloading: 4786it [00:54, 87.36it/s]  
Downloading: 10790it [02:23, 75.06it/s]  
Downloading: 10790it [02:28, 72.42it/s]  
Downloading: 10796it [02:31, 71.36it/s]  
Downloading: 10794it [02:23, 75.30it/s]  
Downloading: 7190it [00:33, 214.82it/s] 
Downloading: 7188it [00:36, 198.07it/s] 
Downloading: 10796it [02:27, 73.23it/s]  
Downloading: 10798it [02:34, 70.03it/s]  
Downloading: 10796it [02:33, 70.40it/s]  
Downloading: 10798it [02:31, 71.18it/s]  
Downloading: 4796it [01:07, 71.18it/s]  
Downloading: 4792it [01:07, 70.76it/s]  


In [18]:
#0,1,2,3,5 artifact (2)
#4,7,8,9 - positive, 6?
blocks = [62, 115, 155, 277, 304, 358, 424, 508, 622, 636]
block_num = blocks[5]
print("Reading output")
fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/wholebrain_results/images/on_" + str(block_num) + "_Probabilities_3d_2channel.h5"
f = h5py.File(fname, "r")
ks = list(f.keys())
pred = f[ks[0]]
pred = pred[1,:,:,:,0]
mask = pred > 0.5
labels = measure.label(mask)
props = measure.regionprops(labels)
for prop in props:
    if prop.area < 100:
        labels[labels == prop.label] = 0


print("Reading input")
fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/wholebrain_results/images/on_" + str(block_num) + ".h5"
f = h5py.File(fname, "r")
ks = list(f.keys())
im = f[ks[0]]
im_bg = im[0,:,:,:,0]
im_fg = im[1,:,:,:,0]

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(im_bg)
viewer.add_image(im_fg)
viewer.add_labels(labels)
napari.run()

Reading output
Reading input


# Make annotation layer

Making new annotation layer
-output data and x,y,z bounds

In [9]:
vol_bg.volume_size

Vec(7403,10240,4800, dtype=int64)

cannot write to https link, can write to s3 link

In [13]:
dir = "s3://smartspim-precomputed-volumes/2021_04_08/gad2cre_tph2flp_con_fon_8291/axon_mask"

info = CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'segmentation',
    data_type       = 'uint64', # Channel images might be 'uint8'
    encoding        = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution      = vol_bg.resolution, # Voxel scaling, units are in nanometers
    voxel_offset    = vol_bg.voxel_offset, # x,y,z offset in voxels from the origin
    # mesh            = 'mesh',
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = [ 128, 128, 2 ], # units are voxels
    volume_size     = vol_bg.volume_size, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(dir, info=info)
vol.commit_info()

In [26]:

image_fg = vol_fg[2432:2560, 3584:3712, 2400:2440]
image_bg = vol_bg[2432:2560, 3584:3712, 2400:2440]
image_2channel = np.stack([image_bg, image_fg], axis=0)
fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/example_annotation/example.h5"
with h5py.File(fname, "w") as f:
    dset = f.create_dataset("image_2channel", data=image_2channel)

Downloading: 50it [00:00, 55.15it/s]
Downloading: 76it [00:00, 79.60it/s]


In [27]:
example = "off_2"

fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/example_annotation/example.h5"
f = h5py.File(fname, 'r')
im = f.get('image_2channel')
im_fg = im[1,:,:,:,0]

fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/example_annotation/example_Probabilities_3d_2channel.h5"
f = h5py.File(fname, 'r')
pred = f.get('exported_data')
pred = pred[1,:,:,:,0]
mask = pred > 0.5
try:
    mask = removeSmallCCs(mask, 100)
except ValueError:
    mask = 0*im_fg

print(im_fg.shape == mask.shape)

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(im_fg)
viewer.add_labels(mask)
napari.run()

True


In [30]:
vol[2432:2560, 3584:3712, 2400:2440] = mask.astype('uint64')

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]




Uploading:   0%|          | 0/1 [00:00<?, ?it/s]





Uploading:   0%|          | 0/1 [00:00<?, ?it/s]






Uploading:   0%|          | 0/1 [00:00<?, ?it/s]







Uploading:   0%|          | 0/1 [00:00<?, ?it/s]








Uploading:   0%|          | 0/1 [00:00<?, ?it/s]









Uploading:   0%|          | 0/1 [00:00<?, ?it/s]










Uploading:   0%|          | 0/1 [00:00<?, ?it/s]











Uploading:   0%|          | 0/1 [00:00<?, ?it/s]












Uploading:   0%|          | 0/1 [00:00<?, ?it/s]













Uploading:   0%|          | 0/1 [00:00<?, ?it/s]














Uploading:   0%|          | 0/1 [00:00<?, ?it/s]















Uploading:   0%|          | 0/1 [00:00<?, ?it/s]
















Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

















Uploading:   0

https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=E-917tNc_GylnQ

## Brainlit example

In [ ]:
dir = "s3://open-neurodata/brainlit/brain2/axons"

info = CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'segmentation',
    data_type       = 'uint64', # Channel images might be 'uint8'
    encoding        = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution      = vol_brainlit.resolution, # Voxel scaling, units are in nanometers
    voxel_offset    = vol_brainlit.voxel_offset, # x,y,z offset in voxels from the origin
    # mesh            = 'mesh',
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = [ 68, 52, 80 ], # units are voxels
    volume_size     = vol_brainlit.volume_size, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(dir, info=info)
vol.commit_info()

In [10]:
import numpy as np 

In [13]:
a = np.zeros((68,52,80), dtype='uint64')
a[30:35,:,:] = 1
vol[:68,:52,:80] = a

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


# Second sample

In [9]:
dir = "precomputed://https://dlab-colm.neurodata.io/2021_06_02_Sert_Cre/Ch_647"

vol_fg = CloudVolume(dir)

## Download and save samples

In [16]:
pos_centers = [[3071, 765, 2342], [5065, 3455, 2342], [3262, 7854, 2342]] #tectum, cortex, olfactory bulb
neg_centers = [[3557, 4797, 2342], [1564, 1997, 2342], [1606, 5204, 2342]] #?, edge of brain/tectum, white matter
radius = 100

for i, center in enumerate(pos_centers):
    image_fg = vol_fg[center[0]-radius:center[0]+radius, center[1]-radius:center[1]+radius, center[2]-radius:center[2]+radius]
    image_bg = 0*image_fg
    image_2channel = np.stack([image_bg, image_fg], axis=0)
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/brain2_data/images/on_" + str(i) + ".h5"
    with h5py.File(fname, "w") as f:
        dset = f.create_dataset("image_2channel", data=image_2channel)


for i, center in enumerate(neg_centers):
    image_fg = vol_fg[center[0]-radius:center[0]+radius, center[1]-radius:center[1]+radius, center[2]-radius:center[2]+radius]
    image_bg = 0*image_fg
    image_2channel = np.stack([image_bg, image_fg], axis=0)
    fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/brain2_data/images/off_" + str(i) + ".h5"
    with h5py.File(fname, "w") as f:
        dset = f.create_dataset("image_2channel", data=image_2channel)

Downloading: 2388it [00:36, 65.59it/s] 
Downloading: 2390it [00:27, 86.18it/s]  
Downloading: 3594it [00:50, 71.85it/s]  


## Read ilastik results

In [8]:
example = "off_2"

fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/brain2_data/images/" + example + ".h5"
f = h5py.File(fname, 'r')
im = f.get('image_2channel')
im_fg = im[1,:,:,:,0]

fname = "/Users/thomasathey/Documents/mimlab/mouselight/ailey/brain2_data/images/" + example + "_Probabilities_3d_2channel.h5"
f = h5py.File(fname, 'r')
pred = f.get('exported_data')
pred = pred[1,:,:,:,0]
mask = pred > 0.5

try:
    mask = removeSmallCCs(mask, 100)
except ValueError:
    mask = 0*im_fg

print(im_fg.shape == mask.shape)

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(im_fg)
viewer.add_labels(mask)
napari.run()

True
